# trainer

> Training loop

In [ ]:
#|default_exp trainer

In [ ]:
#|hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from isaacai.utils import *
from isaacai.dataloaders import *
from isaacai.models import *
from isaacai.initialization import *

from datetime import datetime, timedelta
import torchvision.transforms.functional as TF,torch.nn.functional as F
import math, time

import matplotlib.pyplot as plt
import matplotlib as mpl
import fastcore.all as fc
import torch
from torch import nn, Tensor
from datasets import load_dataset, Dataset
from torch.utils.data import DataLoader
import pandas as pd , numpy as np
from torcheval.metrics import MulticlassAccuracy,Mean
from torch.optim.lr_scheduler import ExponentialLR

import dill as pickle
from fastprogress.fastprogress import master_bar, progress_bar
import inspect
import torchinfo


In [ ]:
#|hide
import logging

In [ ]:
#|hide
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
# mpl.rcParams['image.cmap'] = 'gray'

logging.disable(logging.WARNING)

set_seed(42)


In [ ]:
xmean,xstd = 0.28, 0.35
@inplace
def transformi(b): b['image'] = [(TF.to_tensor(o)-xmean)/xstd for o in b['image']]

_dataset = load_dataset('fashion_mnist').with_transform(transformi)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
_dataset = sample_dataset_dict(_dataset)

In [ ]:
dls = DataLoaders.from_dataset_dict(_dataset, 1024, num_workers=4)

## Base Trainer

In [ ]:
#| export
class CancelFitException(Exception): pass
class CancelBatchException(Exception): pass
class CancelEpochException(Exception): pass

In [ ]:
#| export
cb_steps = ['before_fit','before_epoch', 'before_batch','predict','get_loss','before_backward','backward', 'step',
            'zero_grad','after_batch','cleanup_batch','after_epoch','cleanup_epoch','after_fit','cleanup_fit']

In [ ]:
#| export
def summarize_callbacks(trainer):    
    res = pd.DataFrame(columns=['Step','Callback','Doc String'])
    callbacks = [getattr(trainer,o) for o in trainer.callbacks]
    for attr in cb_steps: 
        for callback in sorted(callbacks, key=lambda x: getattr(x, 'order',0)):
            callback_name = callback.__class__.__name__
            callback = getattr(trainer,callback_name)
            if getattr(callback,attr,None) is not None:
                docstring = getattr(callback,attr).__doc__
                row = pd.DataFrame([[attr,callback_name,fc.ifnone(docstring,'')],],columns=res.columns,index=[''])
                res = pd.concat([res,row])
    return res

In [ ]:
#| export 
class Trainer:
    def subclassing_method(self,**kwargs): pass

    def __init__(self, dls, loss_func, opt_func, model, callbacks, **kwargs):
        self.add_callbacks(callbacks)
        fc.store_attr(but='callbacks')
        self.subclassing_method(**kwargs)
            
    @with_cbs('batch', CancelBatchException)
    def one_batch(self):
        self.run_callbacks(['predict','get_loss'])
        if self.training: self.run_callbacks(['before_backward','backward','step','zero_grad'])
    
    @with_cbs('epoch',CancelEpochException)
    def _one_epoch(self):
        for self.batch_num,self.batch in zip(self.batches,self.dl): self.one_batch()

    def one_epoch(self, training):
        self.model.train(training)
        self.dl = self.dls.train if training else self.dls.valid
        self.batches = range(len(self.dl))
        self._one_epoch()

    @with_cbs('fit', CancelFitException)
    def _fit(self,train, valid):
        for self.epoch in self.epochs: 
            if train: self.one_epoch(True)
            if valid: torch.no_grad()(self.one_epoch)(False)
    
    def fit(self, n_epochs=3, lr=1e-3, callbacks=None,train=True,valid=True):
        fc.store_attr('n_epochs,lr')
        try:
            self.add_callbacks(fc.L(callbacks))
            self.opt = self.opt_func(self.model.parameters(), self.lr)
            self.epochs = range(self.n_epochs)
            self._fit(train,valid)
        finally:
            self.callbacks = [o for o in self.callbacks if o not in fc.L(callbacks)]
                                                        
    @property
    def training(self): return self.model.training
    
    def add_callbacks(self,callbacks,force=False): add_callbacks(self,callbacks,force)

    def run_callbacks(self,method_names):
        cbs = [getattr(self,o) for o in self.callbacks]
        for method_name in fc.L(method_names): run_callbacks(cbs,method_name,self)
            
    def summarize_model(self): return torchinfo.summary(self.model,input_data=fc.first(self.dls.train)[0])
    def summarize_callbacks(self): return summarize_callbacks(self)

In [ ]:
trainer = Trainer(dls,
                  nn.CrossEntropyLoss(), 
                  torch.optim.Adam, 
                  get_model_conv(),
                  callbacks=[])

In [ ]:
trainer.summarize_model()

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [500, 10]                 --
├─Sequential: 1-1                        [500, 8, 14, 14]          --
│    └─Conv2d: 2-1                       [500, 8, 14, 14]          80
│    └─ReLU: 2-2                         [500, 8, 14, 14]          --
├─Sequential: 1-2                        [500, 16, 7, 7]           --
│    └─Conv2d: 2-3                       [500, 16, 7, 7]           1,168
│    └─ReLU: 2-4                         [500, 16, 7, 7]           --
├─Sequential: 1-3                        [500, 32, 4, 4]           --
│    └─Conv2d: 2-5                       [500, 32, 4, 4]           4,640
│    └─ReLU: 2-6                         [500, 32, 4, 4]           --
├─Sequential: 1-4                        [500, 64, 2, 2]           --
│    └─Conv2d: 2-7                       [500, 64, 2, 2]           18,496
│    └─ReLU: 2-8                         [500, 64, 2, 2]           --
├─Seq

## Trainer Summaries

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()